# Lab 3

## Import libs

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statistics as stat
import math
import scipy.stats

## Read data

In [5]:
df = pd.read_excel('/Users/dkravchenko/Jupyter_Notebookes/MathStat/lab3/data.xlsx')
df.head(10)

,reading score,writing score
0,47,32
1,71,21
2,73,35
3,86,36
4,100,10
5,86,10
6,75,54
7,52,35
8,48,55
9,71,10


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype
---  ------         --------------  -----
 0   reading score  200 non-null    int64
 1   writing score  200 non-null    int64
dtypes: int64(2)
memory usage: 3.2 KB


#### Reading scores series

In [7]:
reading_score = df['reading score'].sort_values()

In [8]:
reading_score.describe()

count    200.000000
mean      68.590000
std       15.711338
min       32.000000
25%       56.750000
50%       68.000000
75%       80.250000
max      100.000000
Name: reading score, dtype: float64

#### Writting scores series

In [9]:
writting_score = df['writing score'].sort_values()

In [10]:
writting_score.describe()

count    200.000000
mean      25.065000
std       19.954994
min        0.000000
25%       10.000000
50%       19.000000
75%       34.250000
max      104.000000
Name: writing score, dtype: float64

## Interval series

### Reading interval serie

#### Intervals parameters

In [11]:
n = reading_score.count()
k = int(n ** (1/2))
r = int(reading_score.max() - reading_score.min())
l = r / k

print('---------------------------')
print('|%-8s %5.d|' % ('Number of elements:', n))
print('---------------------------')
print('|%-8s %4.d|' % ('Number of intervals:', k))
print('---------------------------')
print('|%-8s %8.d|' % ('Range of sample:', r))
print('---------------------------')
print('|%-8s %7.1f|' % ('Intervals length:', l))
print('---------------------------')

---------------------------
|Number of elements:   200|
---------------------------
|Number of intervals:   14|
---------------------------
|Range of sample:       68|
---------------------------
|Intervals length:     4.9|
---------------------------


In [24]:
reading_params = {'E': stat.mean(reading_score),
                  'D': stat.variance(reading_score),
                  'Median': stat.median(reading_score),
                  'p_D': stat.pvariance(reading_score),
                  'stdev': stat.stdev(reading_score),
                  'a': stat.mean(reading_score) - (3 * stat.variance(reading_score)) ** (1/2),
                  'b': stat.mean(reading_score) + (3 * stat.variance(reading_score)) ** (1/2)}

for key, value in reading_params.items():
    print('------------------')
    print('|%-8s %7.2f|' % (key + ':', value))
print('------------------')

------------------
|E:         68.59|
------------------
|D:        246.85|
------------------
|Median:    68.00|
------------------
|p_D:      245.61|
------------------
|stdev:     15.71|
------------------
|a:         41.38|
------------------
|b:         95.80|
------------------


#### Table creating

In [57]:
a_i = []
a_i_plus = []

for i in range(k):
    a_i.append(reading_score.min() + l * i)
    a_i_plus.append(reading_score.min() + l * (i + 1))
    
reading_interval = pd.DataFrame({'a_i': a_i,
                                 'a_i+1': a_i_plus})
    
reading_interval.head(k)

,a_i,a_i+1
0,32.000000,36.857143
1,36.857143,41.714286
2,41.714286,46.571429
3,46.571429,51.428571
4,51.428571,56.285714
5,56.285714,61.142857
6,61.142857,66.000000
7,66.000000,70.857143
8,70.857143,75.714286
9,75.714286,80.571429


In [58]:
intervals_values = []
values = reading_score.value_counts().sort_index()

for i in range(k):
    if ((i+1) % 2) and (i != (k - 1)):
        intervals_values.append(sum(values.loc[math.ceil(a_i[i]):math.trunc(a_i_plus[i] - 0.01)]))
    else:
        intervals_values.append(sum(values.loc[math.ceil(a_i[i]):math.trunc(a_i_plus[i])]))

reading_interval['values'] = intervals_values
reading_interval.head(k)

,a_i,a_i+1,values
0,32.000000,36.857143,5
1,36.857143,41.714286,4
2,41.714286,46.571429,5
3,46.571429,51.428571,16
4,51.428571,56.285714,20
5,56.285714,61.142857,14
6,61.142857,66.000000,23
7,66.000000,70.857143,24
8,70.857143,75.714286,18
9,75.714286,80.571429,21


In [59]:
M = []
for value in reading_interval['values']:
    M.append(value / n)
    
F = np.cumsum(M)
reading_interval['F*'] = F
reading_interval.head(k)

,a_i,a_i+1,values,F*
0,32.000000,36.857143,5,0.025
1,36.857143,41.714286,4,0.045
2,41.714286,46.571429,5,0.070
3,46.571429,51.428571,16,0.150
4,51.428571,56.285714,20,0.250
5,56.285714,61.142857,14,0.320
6,61.142857,66.000000,23,0.435
7,66.000000,70.857143,24,0.555
8,70.857143,75.714286,18,0.645
9,75.714286,80.571429,21,0.750


In [60]:
F1 = []
for i in range(k):
    if i < 1:
        F1.append(0)
    elif i > k - 2:
        F1.append(1)
    else:
        F1.append((a_i_plus[i] - reading_params['a']) / (reading_params['b'] - reading_params['a']))

reading_interval['F1'] = F1
reading_interval.head(k)

,a_i,a_i+1,values,F*,F1
0,32.000000,36.857143,5,0.025,0.000000
1,36.857143,41.714286,4,0.045,0.006194
2,41.714286,46.571429,5,0.070,0.095438
3,46.571429,51.428571,16,0.150,0.184681
4,51.428571,56.285714,20,0.250,0.273925
5,56.285714,61.142857,14,0.320,0.363169
6,61.142857,66.000000,23,0.435,0.452412
7,66.000000,70.857143,24,0.555,0.541656
8,70.857143,75.714286,18,0.645,0.630899
9,75.714286,80.571429,21,0.750,0.720143


In [61]:
F2 = []
for i in range(k):
    F2.append((i+1)/k)
    
reading_interval['F2'] = F2
reading_interval.head(k)

,a_i,a_i+1,values,F*,F1,F2
0,32.000000,36.857143,5,0.025,0.000000,0.071429
1,36.857143,41.714286,4,0.045,0.006194,0.142857
2,41.714286,46.571429,5,0.070,0.095438,0.214286
3,46.571429,51.428571,16,0.150,0.184681,0.285714
4,51.428571,56.285714,20,0.250,0.273925,0.357143
5,56.285714,61.142857,14,0.320,0.363169,0.428571
6,61.142857,66.000000,23,0.435,0.452412,0.500000
7,66.000000,70.857143,24,0.555,0.541656,0.571429
8,70.857143,75.714286,18,0.645,0.630899,0.642857
9,75.714286,80.571429,21,0.750,0.720143,0.714286


In [62]:
F_F1 = np.round(np.abs(F - F1), 4)
F_F2 = np.round(np.abs(F - F2), 4)
reading_interval['|F*-F1|'] = F_F1
reading_interval['|F*-F2|'] = F_F2
reading_interval.head(k)

,a_i,a_i+1,values,F*,F1,F2,|F*-F1|,|F*-F2|
0,32.000000,36.857143,5,0.025,0.000000,0.071429,0.0250,0.0464
1,36.857143,41.714286,4,0.045,0.006194,0.142857,0.0388,0.0979
2,41.714286,46.571429,5,0.070,0.095438,0.214286,0.0254,0.1443
3,46.571429,51.428571,16,0.150,0.184681,0.285714,0.0347,0.1357
4,51.428571,56.285714,20,0.250,0.273925,0.357143,0.0239,0.1071
5,56.285714,61.142857,14,0.320,0.363169,0.428571,0.0432,0.1086
6,61.142857,66.000000,23,0.435,0.452412,0.500000,0.0174,0.0650
7,66.000000,70.857143,24,0.555,0.541656,0.571429,0.0133,0.0164
8,70.857143,75.714286,18,0.645,0.630899,0.642857,0.0141,0.0021
9,75.714286,80.571429,21,0.750,0.720143,0.714286,0.0299,0.0357


In [91]:
F1_2 = []
for i in range(k):
    F1_2.append(scipy.stats.norm.cdf(a_i_plus[i], reading_params['E'], reading_params['stdev']))
    
F_F1_2 = np.round(np.abs(F - F1_2), 4)

reading_interval['F1_2'] = F1_2
reading_interval['|F*-F1_2|'] = F_F1_2
reading_interval.head(k)

,a_i,a_i+1,values,F*,F1,F2,|F*-F1|,|F*-F2|,F1_2,|F*-F1_2|
0,32.000000,36.857143,5,0.025,0.000000,0.071429,0.0250,0.0464,0.021705,0.0033
1,36.857143,41.714286,4,0.045,0.006194,0.142857,0.0388,0.0979,0.043578,0.0014
2,41.714286,46.571429,5,0.070,0.095438,0.214286,0.0254,0.1443,0.080541,0.0105
3,46.571429,51.428571,16,0.150,0.184681,0.285714,0.0347,0.1357,0.137352,0.0126
4,51.428571,56.285714,20,0.250,0.273925,0.357143,0.0239,0.1071,0.216770,0.0332
5,56.285714,61.142857,14,0.320,0.363169,0.428571,0.0432,0.1086,0.317751,0.0022
6,61.142857,66.000000,23,0.435,0.452412,0.500000,0.0174,0.0650,0.434531,0.0005
7,66.000000,70.857143,24,0.555,0.541656,0.571429,0.0133,0.0164,0.557368,0.0024
8,70.857143,75.714286,18,0.645,0.630899,0.642857,0.0141,0.0021,0.674887,0.0299
9,75.714286,80.571429,21,0.750,0.720143,0.714286,0.0299,0.0357,0.777148,0.0271


#### Check hypothesis

##### Uniform distribution

In [96]:
quantile = 1.36

def kalmagorov_check(F_f):
    sup = F_f.max()
    if (quantile > sup * (n ** (1/2))):
        return True
    else:
        return False

kalmagorov_check(reading_interval['|F*-F1|']), kalmagorov_check(reading_interval['|F*-F2|'])

(True, False)

###### Conclusion
###### Если брать параметры равномерного распределения исходя из оценок мат. ожидания и дисперсии, то критерий выполняется. В ином случае - не выполняется.

##### Normal distribution

In [97]:
kalmagorov_check(reading_interval['|F*-F1_2|'])

True

###### Coclusion 
###### Гипотеза, что reading score удовлетворяет нормальному распределению принимается.

### Writting interval serie

#### Interval parameters

In [11]:
n = writting_score.count()
k = int(n ** (1/2))
r = int(writting_score.max() - writting_score.min())
l = r / k

print('---------------------------')
print('|%-8s %5.d|' % ('Number of elements:', n))
print('---------------------------')
print('|%-8s %4.d|' % ('Number of intervals:', k))
print('---------------------------')
print('|%-8s %8.d|' % ('Range of sample:', r))
print('---------------------------')
print('|%-8s %7.1f|' % ('Intervals length:', l))
print('---------------------------')

---------------------------
|Number of elements:   200|
---------------------------
|Number of intervals:   14|
---------------------------
|Range of sample:      104|
---------------------------
|Intervals length:     7.4|
---------------------------


In [12]:
writting_params = {'E': stat.mean(writting_score),
                  'D': stat.variance(writting_score),
                  'Median': stat.median(writting_score),
                  'p_D': stat.pvariance(writting_score),
                  'stdev': stat.stdev(writting_score)}

for key, value in writting_params.items():
    print('------------------')
    print('|%-8s %7.2f|' % (key + ':', value))
print('------------------')

------------------
|E:         25.07|
------------------
|D:        398.20|
------------------
|Median:    19.00|
------------------
|p_D:      396.21|
------------------
|stdev:     19.95|
------------------


#### Table creating

In [14]:
a_i = []
a_i_plus = []

for i in range(k):
    a_i.append(writting_score.min() + l * i)
    a_i_plus.append(writting_score.min() + l * (i + 1))
    
writting_interval = pd.DataFrame({'a_i': a_i,
                                 'a_i+1': a_i_plus})
    
writting_interval.head(k)

,a_i,a_i+1
0,0.000000,7.428571
1,7.428571,14.857143
2,14.857143,22.285714
3,22.285714,29.714286
4,29.714286,37.142857
5,37.142857,44.571429
6,44.571429,52.000000
7,52.000000,59.428571
8,59.428571,66.857143
9,66.857143,74.285714


In [15]:
intervals_values = []
values = writting_score.value_counts().sort_index()

for i in range(k):
    if ((i+1) % 2) and (i != (k - 1)):
        intervals_values.append(sum(values.loc[math.ceil(a_i[i]):math.trunc(a_i_plus[i] - 0.01)]))
    else:
        intervals_values.append(sum(values.loc[math.ceil(a_i[i]):math.trunc(a_i_plus[i])]))

writting_interval['values'] = intervals_values
writting_interval.head(k)

,a_i,a_i+1,values
0,0.000000,7.428571,23
1,7.428571,14.857143,54
2,14.857143,22.285714,31
3,22.285714,29.714286,25
4,29.714286,37.142857,28
5,37.142857,44.571429,7
6,44.571429,52.000000,14
7,52.000000,59.428571,4
8,59.428571,66.857143,4
9,66.857143,74.285714,3


In [16]:
c = []
for i in range(k):
    c.append((a_i[i] + a_i_plus[i]) / 2)
writting_interval['c'] = c
writting_interval.head(k)

,a_i,a_i+1,values,c
0,0.000000,7.428571,23,3.714286
1,7.428571,14.857143,54,11.142857
2,14.857143,22.285714,31,18.571429
3,22.285714,29.714286,25,26.000000
4,29.714286,37.142857,28,33.428571
5,37.142857,44.571429,7,40.857143
6,44.571429,52.000000,14,48.285714
7,52.000000,59.428571,4,55.714286
8,59.428571,66.857143,4,63.142857
9,66.857143,74.285714,3,70.571429


In [26]:
p = []
for value in c:
    p.append(3 * np.exp(-1 * writting_params['E']) * (writting_params['E'] ** value) / math.factorial(int(writting_params['E'])))
writting_interval['p'] = p
writting_interval.head(k)

,a_i,a_i+1,values,c,p,ni
0,0.000000,7.428571,23,3.714286,3.957506e-31,7.808364e-09
1,7.428571,14.857143,54,11.142857,9.783539e-21,7.808364e-09
2,14.857143,22.285714,31,18.571429,2.418635e-10,7.808364e-09
3,22.285714,29.714286,25,26.000000,5.979224e+00,7.808364e-09
4,29.714286,37.142857,28,33.428571,1.478152e+11,7.808364e-09
5,37.142857,44.571429,7,40.857143,3.654211e+21,7.808364e-09
6,44.571429,52.000000,14,48.285714,9.033750e+31,7.808364e-09
7,52.000000,59.428571,4,55.714286,2.233276e+42,7.808364e-09
8,59.428571,66.857143,4,63.142857,5.520989e+52,7.808364e-09
9,66.857143,74.285714,3,70.571429,1.364870e+63,7.808364e-09


In [33]:
ni = []
for value in p:
    ni.append(value * n)
writting_interval['ni'] = ni
writting_interval.head(k)

,a_i,a_i+1,values,c,p,ni
0,0.000000,7.428571,23,3.714286,3.957506e-31,7.915012e-29
1,7.428571,14.857143,54,11.142857,9.783539e-21,1.956708e-18
2,14.857143,22.285714,31,18.571429,2.418635e-10,4.837271e-08
3,22.285714,29.714286,25,26.000000,5.979224e+00,1.195845e+03
4,29.714286,37.142857,28,33.428571,1.478152e+11,2.956305e+13
5,37.142857,44.571429,7,40.857143,3.654211e+21,7.308422e+23
6,44.571429,52.000000,14,48.285714,9.033750e+31,1.806750e+34
7,52.000000,59.428571,4,55.714286,2.233276e+42,4.466553e+44
8,59.428571,66.857143,4,63.142857,5.520989e+52,1.104198e+55
9,66.857143,74.285714,3,70.571429,1.364870e+63,2.729740e+65


In [35]:
m_ni = []
for i in range(k):
    m_ni.append(((intervals_values[i] - ni[i])) ** 2 / ni[i])
writting_interval['(m-ni)^2 / ni'] = m_ni
writting_interval.head(k)

,a_i,a_i+1,values,c,p,ni,(m-ni)^2 / ni
0,0.000000,7.428571,23,3.714286,3.957506e-31,7.915012e-29,6.683502e+30
1,7.428571,14.857143,54,11.142857,9.783539e-21,1.956708e-18,1.490258e+21
2,14.857143,22.285714,31,18.571429,2.418635e-10,4.837271e-08,1.986657e+10
3,22.285714,29.714286,25,26.000000,5.979224e+00,1.195845e+03,1.146367e+03
4,29.714286,37.142857,28,33.428571,1.478152e+11,2.956305e+13,2.956305e+13
5,37.142857,44.571429,7,40.857143,3.654211e+21,7.308422e+23,7.308422e+23
6,44.571429,52.000000,14,48.285714,9.033750e+31,1.806750e+34,1.806750e+34
7,52.000000,59.428571,4,55.714286,2.233276e+42,4.466553e+44,4.466553e+44
8,59.428571,66.857143,4,63.142857,5.520989e+52,1.104198e+55,1.104198e+55
9,66.857143,74.285714,3,70.571429,1.364870e+63,2.729740e+65,2.729740e+65


In [48]:
p_2 = []
for value in c:
    p_2.append(7.37 * (1 / writting_params['E']) * np.exp((-1 / writting_params['E']) * value))
writting_interval['p_2'] = p_2
writting_interval.head(k)

,a_i,a_i+1,values,c,p,ni,(m-ni)^2 / ni,p_2
0,0.000000,7.428571,23,3.714286,3.957506e-31,7.915012e-29,6.683502e+30,0.253538
1,7.428571,14.857143,54,11.142857,9.783539e-21,1.956708e-18,1.490258e+21,0.188508
2,14.857143,22.285714,31,18.571429,2.418635e-10,4.837271e-08,1.986657e+10,0.140158
3,22.285714,29.714286,25,26.000000,5.979224e+00,1.195845e+03,1.146367e+03,0.104209
4,29.714286,37.142857,28,33.428571,1.478152e+11,2.956305e+13,2.956305e+13,0.077480
5,37.142857,44.571429,7,40.857143,3.654211e+21,7.308422e+23,7.308422e+23,0.057608
6,44.571429,52.000000,14,48.285714,9.033750e+31,1.806750e+34,1.806750e+34,0.042832
7,52.000000,59.428571,4,55.714286,2.233276e+42,4.466553e+44,4.466553e+44,0.031846
8,59.428571,66.857143,4,63.142857,5.520989e+52,1.104198e+55,1.104198e+55,0.023678
9,66.857143,74.285714,3,70.571429,1.364870e+63,2.729740e+65,2.729740e+65,0.017605


In [50]:
ni_2 = []
for value in p_2:
    ni_2.append(int(value * n))
writting_interval['ni_2'] = ni_2
writting_interval.head(k)

,a_i,a_i+1,values,c,p,ni,(m-ni)^2 / ni,p_2,ni_2
0,0.000000,7.428571,23,3.714286,3.957506e-31,7.915012e-29,6.683502e+30,0.253538,50
1,7.428571,14.857143,54,11.142857,9.783539e-21,1.956708e-18,1.490258e+21,0.188508,37
2,14.857143,22.285714,31,18.571429,2.418635e-10,4.837271e-08,1.986657e+10,0.140158,28
3,22.285714,29.714286,25,26.000000,5.979224e+00,1.195845e+03,1.146367e+03,0.104209,20
4,29.714286,37.142857,28,33.428571,1.478152e+11,2.956305e+13,2.956305e+13,0.077480,15
5,37.142857,44.571429,7,40.857143,3.654211e+21,7.308422e+23,7.308422e+23,0.057608,11
6,44.571429,52.000000,14,48.285714,9.033750e+31,1.806750e+34,1.806750e+34,0.042832,8
7,52.000000,59.428571,4,55.714286,2.233276e+42,4.466553e+44,4.466553e+44,0.031846,6
8,59.428571,66.857143,4,63.142857,5.520989e+52,1.104198e+55,1.104198e+55,0.023678,4
9,66.857143,74.285714,3,70.571429,1.364870e+63,2.729740e+65,2.729740e+65,0.017605,3


In [51]:
m_ni_2 = []
for i in range(k):
    m_ni_2.append(((intervals_values[i] - ni_2[i])) ** 2 / ni_2[i])
writting_interval['(m-ni_2)^2 / ni_2'] = m_ni_2
writting_interval.head(k)

,a_i,a_i+1,values,c,p,ni,(m-ni)^2 / ni,p_2,ni_2,(m-ni_2)^2 / ni_2
0,0.000000,7.428571,23,3.714286,3.957506e-31,7.915012e-29,6.683502e+30,0.253538,50,14.580000
1,7.428571,14.857143,54,11.142857,9.783539e-21,1.956708e-18,1.490258e+21,0.188508,37,7.810811
2,14.857143,22.285714,31,18.571429,2.418635e-10,4.837271e-08,1.986657e+10,0.140158,28,0.321429
3,22.285714,29.714286,25,26.000000,5.979224e+00,1.195845e+03,1.146367e+03,0.104209,20,1.250000
4,29.714286,37.142857,28,33.428571,1.478152e+11,2.956305e+13,2.956305e+13,0.077480,15,11.266667
5,37.142857,44.571429,7,40.857143,3.654211e+21,7.308422e+23,7.308422e+23,0.057608,11,1.454545
6,44.571429,52.000000,14,48.285714,9.033750e+31,1.806750e+34,1.806750e+34,0.042832,8,4.500000
7,52.000000,59.428571,4,55.714286,2.233276e+42,4.466553e+44,4.466553e+44,0.031846,6,0.666667
8,59.428571,66.857143,4,63.142857,5.520989e+52,1.104198e+55,1.104198e+55,0.023678,4,0.000000
9,66.857143,74.285714,3,70.571429,1.364870e+63,2.729740e+65,2.729740e+65,0.017605,3,0.000000


#### Check hypothesis

##### Puasson distribution

In [36]:
sum(writting_interval['(m-ni)^2 / ni'])

1.0195743677254634e+107

In [44]:
scipy.stats.chi2.ppf(1-0.05/2, k-2)

23.33666415864534

###### Coclusion 
###### Гипотеза, что writting score удовлетворяет распределению Пуассона отвергается.

##### Exponential distribution

In [52]:
sum(writting_interval['(m-ni_2)^2 / ni_2'])

45.35011817011817

In [53]:
scipy.stats.chi2.ppf(1-0.05/2, k-2)

23.33666415864534

###### Coclusion 
###### Гипотеза, что writing score удовлетворяет равномерному распределению отвергается.

### Reading and Writting series

#### Interval parameters

In [56]:
n = writting_score.count()
k = int(n ** (1/2))
r = int(writting_score.max() - writting_score.min())
l = r / k

print('---------------------------')
print('|%-8s %5.d|' % ('Number of elements:', n))
print('---------------------------')
print('|%-8s %4.d|' % ('Number of intervals:', k))
print('---------------------------')
print('|%-8s %8.d|' % ('Range of sample:', r))
print('---------------------------')
print('|%-8s %7.1f|' % ('Intervals length:', l))
print('---------------------------')

---------------------------
|Number of elements:   200|
---------------------------
|Number of intervals:   14|
---------------------------
|Range of sample:      104|
---------------------------
|Intervals length:     7.4|
---------------------------


#### Table creating

In [59]:
a_i = []
a_i_plus = []

for i in range(k):
    a_i.append(writting_score.min() + l * i)
    a_i_plus.append(writting_score.min() + l * (i + 1))
    
interval = pd.DataFrame({'a_i': a_i,
                         'a_i+1': a_i_plus})
    
interval.head(k)

,a_i,a_i+1
0,0.000000,7.428571
1,7.428571,14.857143
2,14.857143,22.285714
3,22.285714,29.714286
4,29.714286,37.142857
5,37.142857,44.571429
6,44.571429,52.000000
7,52.000000,59.428571
8,59.428571,66.857143
9,66.857143,74.285714


In [60]:
reading_values = []
writting_values = []
r_values = reading_score.value_counts().sort_index()
w_values = writting_score.value_counts().sort_index()

for i in range(k):
    if ((i+1) % 2) and (i != (k - 1)):
        reading_values.append(sum(r_values.loc[math.ceil(a_i[i]):math.trunc(a_i_plus[i] - 0.01)]))
        writting_values.append(sum(w_values.loc[math.ceil(a_i[i]):math.trunc(a_i_plus[i] - 0.01)]))
    else:
        reading_values.append(sum(r_values.loc[math.ceil(a_i[i]):math.trunc(a_i_plus[i])]))
        writting_values.append(sum(w_values.loc[math.ceil(a_i[i]):math.trunc(a_i_plus[i])]))

interval['r_M'] = reading_values
interval['w_M'] = writting_values
interval.head(k)

,a_i,a_i+1,r_M,w_M
0,0.000000,7.428571,0,23
1,7.428571,14.857143,0,54
2,14.857143,22.285714,0,31
3,22.285714,29.714286,0,25
4,29.714286,37.142857,5,28
5,37.142857,44.571429,8,7
6,44.571429,52.000000,17,14
7,52.000000,59.428571,29,4
8,59.428571,66.857143,32,4
9,66.857143,74.285714,36,3


In [62]:
r_M = []
w_M = []
for i in range(k):
    r_M.append(interval['r_M'][i] / n)
    w_M.append(interval['w_M'][i] / n)
    
r_F = np.cumsum(r_M)
w_F = np.cumsum(w_M)
interval['r_F*'] = r_F
interval['w_F*'] = w_F
interval.head(k)

,a_i,a_i+1,r_M,w_M,r_F*,w_F*
0,0.000000,7.428571,0,23,0.000,0.115
1,7.428571,14.857143,0,54,0.000,0.385
2,14.857143,22.285714,0,31,0.000,0.540
3,22.285714,29.714286,0,25,0.000,0.665
4,29.714286,37.142857,5,28,0.025,0.805
5,37.142857,44.571429,8,7,0.065,0.840
6,44.571429,52.000000,17,14,0.150,0.910
7,52.000000,59.428571,29,4,0.295,0.930
8,59.428571,66.857143,32,4,0.455,0.950
9,66.857143,74.285714,36,3,0.635,0.965


In [66]:
interval['|r_F* - w_F*|'] = np.round(np.abs(r_F - w_F), 4)
interval.head(k)

,a_i,a_i+1,r_M,w_M,r_F*,w_F*,|r_F* - w_F*|
0,0.000000,7.428571,0,23,0.000,0.115,0.115
1,7.428571,14.857143,0,54,0.000,0.385,0.385
2,14.857143,22.285714,0,31,0.000,0.540,0.540
3,22.285714,29.714286,0,25,0.000,0.665,0.665
4,29.714286,37.142857,5,28,0.025,0.805,0.780
5,37.142857,44.571429,8,7,0.065,0.840,0.775
6,44.571429,52.000000,17,14,0.150,0.910,0.760
7,52.000000,59.428571,29,4,0.295,0.930,0.635
8,59.428571,66.857143,32,4,0.455,0.950,0.495
9,66.857143,74.285714,36,3,0.635,0.965,0.330


#### Check hypothesis

In [67]:
quantile = 1.36

def homogeneity_check(F_f):
    sup = F_f.max()
    if (quantile > sup * ((n/2) ** (1/2))):
        return True
    else:
        return False

homogeneity_check(interval['|r_F* - w_F*|'])

False

###### Coclusion 
###### Гипотеза, что reading score и writing score однородны (имеют одинаковое распределение) отвергается.

### Выполнял: Кравченко Д. В.
### Группа R3338
### Вариант №3